## Imports

In [1]:
import json
import os
from datetime import timedelta, datetime
from time import sleep
from sys import argv
import gensim
import gensim.corpora as corpora
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import torch_geometric
from functools import reduce
from tqdm import tqdm
from Preprocessing import Preprocessor
preprocessor = Preprocessor(0)
tqdm.pandas()

c:\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
c:\Python310\lib\site-packages\joblib\backports.py:7: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


## Load the metadata and transcripts and join them together

In [2]:
ted_main = pd.read_csv("Datasets\\TEDTalksDataset\\ted_main.csv")
transcripts = pd.read_csv("Datasets\\TEDTalksDataset\\transcripts.csv")
ted = ted_main.join(transcripts, lsuffix='url', rsuffix='url', sort=True)
ted = ted.dropna()
ted.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2466
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2461 non-null   int64 
 1   description         2461 non-null   object
 2   duration            2461 non-null   int64 
 3   event               2461 non-null   object
 4   film_date           2461 non-null   int64 
 5   languages           2461 non-null   int64 
 6   main_speaker        2461 non-null   object
 7   name                2461 non-null   object
 8   num_speaker         2461 non-null   int64 
 9   published_date      2461 non-null   int64 
 10  ratings             2461 non-null   object
 11  related_talks       2461 non-null   object
 12  speaker_occupation  2461 non-null   object
 13  tags                2461 non-null   object
 14  title               2461 non-null   object
 15  urlurl              2461 non-null   object
 16  views               2461

In [3]:
ted.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,urlurl,views,transcript,urlurl
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


## Clean up the transcripts using the preprocessor

In [4]:
ted['transcript_cleaned'] = ted['transcript'].progress_apply(lambda x: preprocessor.clean(x, fast=True))
print(ted['transcript_cleaned'][0][0:200])

 81%|████████  | 1985/2461 [00:09<00:01, 238.11it/s]c:\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|██████████| 2461/2461 [00:11<00:00, 218.68it/s]

good morning. you? it great, it? blown away whole thing. fact, leaving. there three themes running conference relevant want talk about. one extraordinary evidence human creativity presentations people


## Tokenize the transcript using the preprocessor

In [5]:
# clear up the lists to be unique only.
ted['tokens'] = ted['transcript'].progress_apply(lambda x: preprocessor.clean(x, tokenize=True, fast=True))
print(len(ted['tokens'][0]))
print(ted['tokens'][0][0:100])

 81%|████████  | 1993/2461 [00:09<00:01, 239.49it/s]c:\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|██████████| 2461/2461 [00:11<00:00, 216.95it/s]

1484
['good', 'morning.', 'you?', 'it', 'great,', 'it?', 'blown', 'away', 'whole', 'thing.', 'fact,', 'leaving.', 'there', 'three', 'themes', 'running', 'conference', 'relevant', 'want', 'talk', 'about.', 'one', 'extraordinary', 'evidence', 'human', 'creativity', 'presentations', 'people', 'here.', 'variety', 'range', 'it.', 'second', 'put', 'us', 'place', 'idea', 'going', 'happen,', 'terms', 'future.', 'idea', 'may', 'play', 'out.i', 'interest', 'education.', 'actually,', 'find', 'everybody', 'interest', 'education.', 'you?', 'find', 'interesting.', 'dinner', 'party,', 'say', 'work', 'educationactually,', 'often', 'dinner', 'parties,', 'frankly.', 'if', 'work', 'education,', 'asked.', 'and', 'never', 'asked', 'back,', 'curiously.', 'strange', 'me.', 'are,', 'say', 'somebody,', 'know,', 'say,', 'what', 'do?', 'say', 'work', 'education,', 'see', 'blood', 'run', 'face.', 'like,', 'oh', 'god,', 'know,', 'why', 'me?', 'my', 'one', 'night', 'week.', 'but']


In [6]:
ted['transcript_n_entries'] = ted['tokens'].progress_apply(lambda x: len(x))
ted.head(5)

100%|██████████| 2461/2461 [00:00<00:00, 738195.10it/s]


,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,...,speaker_occupation,tags,title,urlurl,views,transcript,urlurl,transcript_cleaned,tokens,transcript_n_entries
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,...,Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,"good morning. you? it great, it? blown away wh...","[good, morning., you?, it, great,, it?, blown,...",1484
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,...,Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,"thank much, chris. truly great honor opportuni...","[thank, much,, chris., truly, great, honor, op...",1012
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,...,Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,"(music: the sound silence, simon & garfunkel)h...","[(music:, the, sound, silence,, simon, &, garf...",1673
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,...,Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,todayhappyheard sustainable development save u...,"[todayhappyheard, sustainable, development, sa...",1624
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,...,Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,"10 years ago, took task teach global developme...","[10, years, ago,, took, task, teach, global, d...",1535


## Save the results to a local file to speed up rerunning further documents

In [7]:
ted.to_csv("ted_joined.csv", index=False)

## Train a simple unsupervised LDA model on the transcripts

### First we need to figure out how many topics we're interested in.
This can be extracted from the 'tags' feature. This feature functions as the keywords that are relevant for each video.

In [8]:
ted['tags'][0].replace('[', '').replace(']', '').replace('\'', '').split(', ')

['children',
 'creativity',
 'culture',
 'dance',
 'education',
 'parenting',
 'teaching']

In [9]:
topics = []
for video in tqdm(ted['tags']):
    tags = video.replace('[', '').replace(']', '').replace('\'', '').split(', ')
    topics += tags
topics = list(set(topics))
print(topics[0:10])
print(len(topics))

100%|██████████| 2461/2461 [00:00<00:00, 307592.29it/s]

['microsoft', 'literature', 'sex', 'open-source', 'energy', 'AI', 'collaboration', 'Transgender', 'marketing', 'ecology']
415


415 is a lot of topics. There is probably a lot of overlap in these, so lets knock it down to just 10% of that number.

In [10]:
num_topics = len(topics) // 10
num_topics

41

In [11]:
print(type(ted['tokens'][0]))

<class 'list'>


In [12]:
%%time
# LDA Normal
# temp = [d.split() for d in corpus]
try:
    temp = [doc.split() for doc in ted['tokens']]
except AttributeError:
    temp = ted['tokens']
print(type(temp))
words = corpora.Dictionary(temp)
words.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
corpus = [words.doc2bow(doc) for doc in temp]

<class 'pandas.core.series.Series'>
CPU times: total: 3.77 s
Wall time: 3.81 s


In [13]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=num_topics,
                                           random_state=2,
                                           update_every=1,
                                           passes=15,
                                           alpha='auto')

CPU times: total: 5min 45s
Wall time: 4min 47s


## Prepare a visualization of the LDA model for investigation

In [14]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, words, mds='mmds', R=10)

c:\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [15]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
36     0.001074 -0.013930       1        1  14.419450
40     0.037163  0.076610       2        1  14.260499
29    -0.062002  0.003009       3        1  12.841551
32    -0.028746  0.057890       4        1  11.150276
8      0.156562  0.072697       5        1   7.963366
26     0.089681  0.029461       6        1   7.753152
37    -0.002634  0.158403       7        1   4.424234
35    -0.151040  0.040484       8        1   4.005865
20    -0.112668  0.115556       9        1   3.622104
12     0.078162  0.232372      10        1   2.975564
23     0.157149  0.147454      11        1   2.262776
6      0.023894  0.324808      12        1   2.243646
24    -0.307072  0.276302      13        1   1.541467
9     -0.089696  0.236511      14        1   1.424139
11    -0.221126  0.218258      15        1   1.416108
1     -0.039520  0.419264      16        1   1.293611
28    -0.218345  0.389455      17        1   0.856305
31    -0.407941  0.145607      18        1   0.751513
33     0.217432  0.292950      19        1   0.689572
4      0.368109  0.224321      20        1   0.607680
27    -0.407163  0.011935      21        1   0.577876
22     0.297471  0.052116      22        1   0.569201
34     0.230827  0.395469      23        1   0.525087
16    -0.376646 -0.302400      24        1   0.376152
2     -0.402125 -0.143380      25        1   0.285102
25     0.410799  0.050175      26        1   0.275789
0      0.476313 -0.137768      27        1   0.250639
13    -0.298594  0.012879      28        1   0.245367
18     0.352586 -0.283526      29        1   0.150839
21    -0.098969 -0.470752      30        1   0.090866
39    -0.032625 -0.293026      31        1   0.034944
14     0.092235 -0.349065      32        1   0.032984
15    -0.117819 -0.292616      33        1   0.027647
10     0.096452 -0.261168      34        1   0.018114
19     0.036159 -0.245480      35        1   0.014329
7      0.093771 -0.223524      36        1   0.014151
38     0.031778 -0.193465      37        1   0.001620
3      0.031778 -0.193470      38        1   0.001609
30     0.031778 -0.193471      39        1   0.001605
17     0.031778 -0.193472      40        1   0.001604
5      0.031778 -0.193472      41        1   0.001601, topic_info=         Term         Freq        Total Category  logprob  loglift
344      love  2060.000000  2060.000000  Default  10.0000  10.0000
660     women  1734.000000  1734.000000  Default   9.0000   9.0000
82      brain  1610.000000  1610.000000  Default   8.0000   8.0000
1066   wanted  2261.000000  2261.000000  Default   7.0000   7.0000
278     human  3701.000000  3701.000000  Default   6.0000   6.0000
...       ...          ...          ...      ...      ...      ...
318     know,     0.001894  2833.071592  Topic41  -9.5159  -3.1760
280      idea     0.001893  2466.788105  Topic41  -9.5161  -3.0378
596     this,     0.001893  2084.564152  Topic41  -9.5162  -2.8696
1420  percent     0.001893  2918.377709  Topic41  -9.5162  -3.2060
339      long     0.001893  2192.776912  Topic41  -9.5162  -2.9202

[894 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
13608      1  0.985495  2016
7650       1  0.044782    97
7650       2  0.134345    97
7650      16  0.268691    97
7650      25  0.492600    97
...      ...       ...   ...
3499       1  0.453532  zone
3499       6  0.009859  zone
3499      12  0.029578  zone
3499      14  0.059156  zone
3499      15  0.443673  zone

[5102 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[37, 41, 30, 33, 9, 27, 38, 36, 21, 13, 24, 7, 25, 10, 12, 2, 29, 32, 34, 5, 28, 23, 35, 17, 3, 26, 1, 14, 19, 22, 40, 15, 16, 11, 20, 8, 39, 4, 31, 18, 6])

## Run a simple prediction as a test

In [16]:
new_text_corpus = words.doc2bow(ted['transcript'][0].split())
print(len(new_text_corpus))
prediction = lda_model.get_document_topics(new_text_corpus)
prediction.sort(key = lambda x: x[1], reverse = True)
print(prediction)
prediction = prediction[0][0]
print("Predicted Topic: %d" % prediction)
lda_model.show_topic(prediction)

657
[(29, 0.69394505), (37, 0.10825403), (36, 0.10349321), (8, 0.051284358), (40, 0.026880337)]
Predicted Topic: 29


[('i', 0.004483086),
 ('love', 0.004303254),
 ('me,', 0.0042840424),
 ('feel', 0.0039194655),
 ('know,', 0.0034427464),
 ('story', 0.0032321173),
 ('always', 0.0031905267),
 ('day', 0.0031634443),
 ('said', 0.0031312134),
 ('wanted', 0.0030878563)]